# Gleam
This notebook aims to design, train, validate, and use a convolutional neural network that is able to predict the distribution of population in a geographical area from nighttime satellite imagery. The product is a high resolution approximation map of the population distribution.

## Datasets

[Recommended source for nighttime pictures (NOAA)](https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html)

[Recommended source for population rasters (SEDAC)](http://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-rev10)

[Recommended vector file to isolate a country (Natural Earth)](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/)

This data needs to be processed with a GIS tool to meet a few requirements before training :
- Rasters of the same year need to be merged in one GeoTIFF file : lights on the first band, population on the second band.
- Since both sets don't have the same resolution, upscaling the population raster might have increased population density. This needs to be adjusted before training (use `rescale_pop.py` in the same folder as this notebook, don't forget to change some variables). In the case of the recommended datasets, every population pixel needs to be devided by 4 after merging with gdal_merge or QGIS.
- Use a vector file to clip the raster and isolate the region of interest. Areas with a population of zero (out of borders or in the ocean) will be ignored during preprocessing.

## Trained models
A few already trained models can be found in `./models`. These were trained on the recommended datasets using the scripts below for the year 2015, so the input dataset for a prediction must have 1 pixel per 0.25 square kilometer. The name of the model indicates the contries that were used to train it (`safrica_namibia.h5` is South Africa and Namibia).

## Imports

In [ ]:
import rasterio
import numpy as np
import keras.layers.core as core
import keras.layers.convolutional as conv
import keras.models as models
import keras.callbacks
from sklearn.model_selection import KFold
from keras import optimizers
import time

## Preprocessing
The preprocess function creates a sliding window over the input raster.

In [ ]:
def preprocess(filepath, input_tile_size, offset):
    """
    :param filepath: GeoTIFF raster file with 2 bands
    :param input_tile_size: width of the square sliding window, and of the output tiles
    :param offset: distance in pixels the window will move between each tile
    :return: (X, Y) where X is a suitable array of inputs for the neural network
            and Y is the expected output for each of these inputs
    This function filters out the tiles that have zero population on them.
    """
    
    raster = rasterio.open(filepath)

    matrix_x = raster.read(1)
    matrix_y = raster.read(2)

    X = []
    Y = []
    col = 0
    while col + input_tile_size < matrix_x.shape[1]:
        row = 0
        while row + input_tile_size < matrix_x.shape[0]:
            pop = np.sum(matrix_y[row: row + input_tile_size, col: col + input_tile_size])
            # only use tiles that have people living on it
            if pop > 0:
                X.append(matrix_x[row: row + input_tile_size, col: col + input_tile_size])
                Y.append(pop)

            row += offset
        col += offset

    raster.close()
    matrix_x, matrix_y = None, None  # free some memory
    X, Y = np.array(X), np.array(Y)
    X = np.expand_dims(X, axis=3)  # add the color channel as a new dimension
    print('input shape (observations, obs_width, obs_height, channels) : ' + str(X.shape))
    return X, Y


## The neural network
This part of the script defines how the neural network will be initialized and creates callbacks.

In [ ]:
def init_cnn(input_shape):
    # kernel size for each convolution layer
    kernel_size = (3, 3)
    
    cnn = models.Sequential()

    cnn.add(conv.Convolution2D(filters=64, kernel_size=kernel_size, activation="relu", padding='same',
                               input_shape=input_shape))
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))
    
    cnn.add(conv.Convolution2D(filters=128, kernel_size=kernel_size, activation="relu", padding='same'))
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))

    cnn.add(conv.Convolution2D(filters=256, kernel_size=kernel_size, activation="relu", padding='same'))
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))
    
    cnn.add(core.Flatten())
    cnn.add(core.Dropout(0.5))
    cnn.add(core.Dense(128))
    cnn.add(core.Dense(1))

    cnn.compile(loss="mean_squared_error", optimizer=optimizers.Adam(lr=0.02, decay=0.0), metrics=["mse", "mae"])
    return cnn

# CALLBACKS
# reduce learning rate when we stopped learning anything
rlrp = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=20, verbose=1, mode='auto', min_lr=0.0000001)

# stop learning early if we stopped leaning anything for a longer time
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=200, verbose=1, mode='auto')

## K-Fold validation
This script trains and validates the model using 4-fold cross-validation. It was used to find the best performing neural network topology and measure the amount of error we can expect from a trained model. The resulting model is saved to the models/ subfolder.

In [ ]:
## PARAMETERS ##
nb_epoch = 2000  # maximum number of epochs
model_birthday = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())  # used to identify generated files (logs and models)
raster_path = '../data/lightpop_merged/adj_2015_usa.tif'  # raster containing the training dataset
verbose = 2  # 0: no progress message, 1: progress bar for each epoch, 2: one message for each epoch
################

# CALLBACKS
# logs for tensorboard
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/" + model_birthday)

print('preprocessing')
X, Y = preprocess(raster_path, 32, 32)

print('configuring cnn')

# input dimensions
img_count, img_rows, img_cols, img_channel_count = X.shape

# k-fold split
kfold = KFold(n_splits=4, shuffle=True, random_state=None)

# initialize statistics aggregate
kfold_mse = []
kfold_mae = []
kfold_sae = []

print('logs will be saved to logs/' + model_birthday)

for train, test in kfold.split(X, Y):
    
    cnn = init_cnn((img_rows, img_cols, img_channel_count))

    print('training ...')

    cnn.fit(X[train], Y[train], batch_size=1024, epochs=nb_epoch, verbose=verbose,
            callbacks=[tensorboard, rlrp, early_stopping],
            sample_weight=None)

    cnn.save('models/' + model_birthday + '.h5')

    print('model saved to models/' + model_birthday + '.h5')
    
    # evaluate and print stats
    evaluation = cnn.evaluate(X[test], Y[test], verbose=2, batch_size=1024)
    evaluation = dict(zip(cnn.metrics_names, evaluation))
    kfold_mse.append(evaluation['mean_squared_error'])
    kfold_mae.append(evaluation['mean_absolute_error'])
    kfold_sae.append(evaluation['mean_absolute_error'] * len(Y[test]))
    
    print('K-fold validation results :')
    print('Mean squared error : %.2f (std %.2f)' % (np.mean(kfold_mse), np.std(kfold_mse)))
    print('Mean absolute error : %.2f (std %.2f)' % (np.mean(kfold_mae), np.std(kfold_mae)))
    print('Sum of absolute errors : %.2f (std %.2f)' % (np.mean(kfold_sae), np.std(kfold_sae)))

print('done !')


## Training without validation
Trains on a whole raster once, without validating. The goal is to build a model that makes the best possible prediction, so it has to train on every data available including the validation set. The resulting model is saved to the `./models` subfolder.

In [ ]:
## PARAMETERS ##
nb_epoch = 5000  # maximum number of epochs
model_birthday = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())  # used to identify generated files (logs and models)
raster_path = '../data/lightpop_merged/adj_2015_usa.tif'  # raster containing the training dataset
verbose = 2  # 0: no progress message, 1: progress bar for each epoch, 2: one message for each epoch
################

# CALLBACKS
# logs for tensorboard
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/" + model_birthday)

# checkpoints in case of crash or interruption
checkpoint = keras.callbacks.ModelCheckpoint('models/' + model_birthday + '.h5', save_weights_only=False)

print('preprocessing')

X, Y = preprocess(raster_path, 32, 8)

print('configuring cnn')

# input dimensions
img_count, img_rows, img_cols, img_channel_count = X.shape

print('logs will be saved to logs/' + model_birthday)

cnn = init_cnn((img_rows, img_cols, img_channel_count))

print('training ...')

cnn.fit(X, Y, batch_size=1024, epochs=nb_epoch, verbose=verbose,
        callbacks=[tensorboard, checkpoint, rlrp, early_stopping], sample_weight=None)

cnn.save('models/' + model_birthday + '.h5')

print('model saved to models/' + model_birthday + '.h5')

print('done !')

## Prediction
This script generates a high resolution population raster from a satellite image and a trained model. It uses its own preprocessing algorithm that works without validation data.

Since the model gives only one output for each 32x32 tile, the distribution of the output value in the tile is managed by this script. It assumes a log relation between nightlights and population for each pixel.

The notebook "rastercomparator" can then be used to compare actual data with predicted data, as well as the predicted population counts over the years.

In [ ]:
## PARAMETERS ##
model_path = "models/usa.h5"
prediction_dataset = '../data/lightpop_merged/adj_2015_portugal.tif'  # input nightlights
out = '2015_brazil_to_2015_portugal.tif'  # output population file
input_tile_size = 32
################

print('loading model')

cnn = models.load_model(model_path)

print('opening raster')

raster = rasterio.open(prediction_dataset)
band = raster.read(1)
profile = raster.profile
profile.update(count=1)
width, height = raster.width, raster.height

# preprocess
matrix_x = raster.read(1)
tiles_x = []
y = 0
while y + input_tile_size < matrix_x.shape[1]:
    x = 0
    while x + input_tile_size < matrix_x.shape[0]:
        tiles_x.append(matrix_x[x: x + input_tile_size, y: y + input_tile_size])
        x += input_tile_size
    y += input_tile_size
testX = np.array(tiles_x)
raster.close()
matrix_x = None
tiles_x = None
testX = np.expand_dims(testX, axis=3)

print('generating raster')

predicted_tiles = cnn.predict(testX, verbose=0)

# rebuild raster from predictions
predicted_raster = np.zeros(shape=(raster.height, raster.width))
y = 0
pred_index = 0
while y + input_tile_size < width:
    x = 0
    while x + input_tile_size < height:
        in_tile = band[x: x + input_tile_size, y: y + input_tile_size]
        if np.max(in_tile) <= 0:
            # avoid divisions by 0
            predicted_raster[x: x + input_tile_size, y: y + input_tile_size] = 0
        else:
            # normalize visible light between 0 and 1 to avoid overflows (also gives better results)
            weights = in_tile / np.max(in_tile)
            # visible light is perceived logarithmically => counteract with exp
            weights = np.exp(weights) - 1
            # the sum of all weights must be 1
            weights = weights / np.sum(weights)
            predicted_raster[x: x + input_tile_size, y: y + input_tile_size] = predicted_tiles[pred_index] * weights

        pred_index += 1
        x += input_tile_size
    y += input_tile_size

predicted_raster = np.array(predicted_raster)

with rasterio.open('predictions/' + out, 'w', **profile) as dst:
    dst.write(predicted_raster.astype(rasterio.float32), 1)
print("prediction saved to predictions/" + out)

predicted_raster = None

print('prediction done !')
